# Práctica 1: Clasificación multi-etiqueta

## Ejercicio 5

Familiarízate con la documentación scikit-multilearn en: http://scikit.ml/ Prueba los métodos ML disponibles pertenecientes a las dos categorías (transformación y adaptación) que hemos visto en teoría.

### Enfoque

Para realizar este apartado se va a tomar el conjunto de datos `scene` ya que ha sido uno de los conjuntos de datos que menos tiempo ha llevado procesar en los apartados anteriores, siendo un conjunto de datos no muy grande pero tampoco pequeño. Lo que lo hace ideal para realizar pruebas sin que lleven un tiempo excesivo. Sus características, obtenidas en apartados anteriores, son las siguientes:

- Número de instancias en scene: 2407
- Número de atributos en scene: 294
- Número de etiquetas en scene: 6
- Car en scene: 1.0739509763190693
- Den en scene: 0.17899182938651156
- Div en scene: 0.234375
- AvgIR en scene: 1.2537840597429668
- rDep en scene: 0.9333333333333333

In [ ]:
# Due to the error described here: https://stackoverflow.com/questions/69734051/mlknn-int-takes-1-positional-argument-but-2-were-given-with-fit-method
# And experimented during implementation, deprecated package is needed:
# Explained at the adaptation part of the notebook
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.24.1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from skmultilearn.dataset import load_dataset

# Transform
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from skmultilearn.ensemble import RakelO, RakelD

# Adaptation
from skmultilearn.adapt import MLkNN, BRkNNaClassifier, MLTSVM

from sklearn.naive_bayes import GaussianNB

# Metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, hamming_loss

Scikit-multilearn permite importar los conjuntos de datos ya divididos en conjuntos de entrenamiento y test, por ello, se van a cargar directamente estas divisiones.

Esto se va a realizar así ya que al realizar el split entre train y test hay que tener muchos datos de la distribución del conjunto de datos en cuenta, como el número de instancias, el número de instancias por clase... . Al estar ya divididos se entiende que se han tenido en cuetna estos datos y se han realizado conjuntos de datos de entrenamiento y test independientes y fiables para la evaluación de las métricas de predicción posteriormente, evitando resultados demasiados optimistas o pesimistas

In [2]:
dataset_name = "scene"
X_train, y_train, feature_names_train, label_names_train = load_dataset(dataset_name, 'train')
X_test, y_test, feature_names_test, label_names_test = load_dataset(dataset_name, 'test')

scene:train - exists, not redownloading
scene:test - exists, not redownloading


### Transformación

Hay algoritmos que necesitan tener un clasificador base, se ha seleccioando un único clasificador para utilizar como base, que es GaussianNB. Para todos los algoritmos se utilizará este algoritmo como clasificador base para intentar hacer así una mejor comparación. Aún así, es importante tener en cuenta que, quizás con otro clasificador base haya algoritmos que obtengan mejores resultados, ya que las combinaciones pueden ser muy diversas.

#### Binary Relevance

Este método convierte un problema de clasificación multietiqueta en varios problemas de clasificación binaria independientes, uno para cada etiqueta. Utiliza el clasificador Naive Bayes gaussiano (GaussianNB) como clasificador base.

In [14]:
classifier = BinaryRelevance(GaussianNB())
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
accuracy_binary_relevance = accuracy_score(y_test, predictions)
f1_binary_relevance = f1_score(y_test, predictions, average='micro')
precision_binary_relevance = precision_score(y_test, predictions, average='micro')

print("Binary Relevance con GaussianNB:")
print("Accuracy =", accuracy_binary_relevance)
print("F1 =", f1_binary_relevance)
print("Precision =", precision_binary_relevance)

Binary Relevance con GaussianNB:
Accuracy = 0.17809364548494983
F1 = 0.5503217503217502
Precision = 0.4133797370456303


#### Classifier Chain

Classifier Chain extiende la idea de Binary Relevance conectando clasificadores binarios en cadena, donde cada clasificador toma las características de entrada y las predicciones de todas las etiquetas anteriores. Usa GaussianNB como clasificador base.

In [5]:
classifier = ClassifierChain(GaussianNB())
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
accuracy_classifier_chain = accuracy_score(y_test, predictions)
f1_classifier_chain = f1_score(y_test, predictions, average='micro')
precision_classifier_chain = precision_score(y_test, predictions, average='micro')

print("\nClassifier Chain con GaussianNB:")
print("Accuracy =", accuracy_classifier_chain)
print("F1 =", f1_classifier_chain)
print("Precision =", precision_classifier_chain)


Classifier Chain con GaussianNB:
Accuracy = 0.2909698996655518
F1 = 0.5266177876952503
Precision = 0.44940152339499456


#### Label Powerset

Label Powerset transforma el problema de clasificación multietiqueta en un problema de clasificación multiclase, asignando a cada combinación única de etiquetas una clase. Utiliza GaussianNB como clasificador base.

In [6]:
classifier = LabelPowerset(GaussianNB())
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
accuracy_label_powerset = accuracy_score(y_test, predictions)
f1_label_powerset = f1_score(y_test, predictions, average='micro')
precision_label_powerset = precision_score(y_test, predictions, average='micro')

print("\nLabel Powerset with GaussianNB:")
print("Accuracy =", accuracy_label_powerset)
print("F1 =", f1_label_powerset)
print("Precision =", precision_label_powerset)


Label Powerset with GaussianNB:
Accuracy = 0.5560200668896321
F1 = 0.6304098046725393
Precision = 0.6272865853658537


#### Ensemble methods (RakelO)

RakelO es un método de ensemble que divide las etiquetas en subconjuntos y entrena un clasificador binario para cada subconjunto. Combina las predicciones de los clasificadores. Usa GaussianNB como clasificador base.

In [7]:
classifier = RakelO(base_classifier=GaussianNB(), model_count=3)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
accuracy_rakelO = accuracy_score(y_test, predictions)
f1_rakelO = f1_score(y_test, predictions, average='micro')
precision_rakelO = precision_score(y_test, predictions, average='micro')

print("\nRakelO with GaussianNB:")
print("Accuracy =", accuracy_rakelO)
print("F1 =", f1_rakelO)
print("Precision =", precision_rakelO)


RakelO with GaussianNB:
Accuracy = 0.3311036789297659
F1 = 0.618001855861429
Precision = 0.516546018614271


### Adaptación

Al realizar el apartado de adaptación se han experimentado problemas con la librería, tras intentar encontrar soluciones en el preprocesamiento de los datos, al buscar el problema en internet se han encontrado a usuarios con el mismo problema y que han tenido que utilizar una versión anterior de la librería scikit-multilearn. Por eso, para este notebook en concreto se está utilizando una versión anterior, indicada en las primeras celdas.

#### MLkNN

MLkNN es un método de adaptación basado en k-vecinos más cercanos que ajusta su modelo a los datos de entrenamiento y predice las etiquetas de las instancias de prueba utilizando la información de los k-vecinos más cercanos. En este caso, no se utiliza un clasificador base adicional.

In [8]:
classifier = MLkNN(k=10)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
accuracy_mlknn = accuracy_score(y_test, predictions)
f1_mlknn = f1_score(y_test, predictions, average='micro')
precision_mlknn = precision_score(y_test, predictions, average='micro')

print("\nMLkNN:")
print("Accuracy =", accuracy_mlknn)
print("F1 =", f1_mlknn)
print("Precision =", precision_mlknn)

c:\Users\albam\anaconda3\envs\MachineLearning\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "



MLkNN with GaussianNB:
Accuracy = 0.6053511705685619
F1 = 0.7227467811158798
Precision = 0.8166828322017459


#### BRkNNaClassifier
BRkNNaClassifier es un clasificador basado en k-vecinos más cercanos que se adapta a problemas multietiqueta.

In [9]:
classifier = BRkNNaClassifier(k=3)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
accuracy_brknn = accuracy_score(y_test, predictions)
f1_brknn = f1_score(y_test, predictions, average='micro')
precision_brknn = precision_score(y_test, predictions, average='micro')

print("\nBRkNNaClassifier:")
print("Accuracy =", accuracy_brknn)
print("F1 =", f1_brknn)
print("Precision =", precision_brknn)

c:\Users\albam\anaconda3\envs\MachineLearning\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=3 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "



BRkNNaClassifier with GaussianNB:
Accuracy = 0.6003344481605352
F1 = 0.6779521056977704
Precision = 0.7310774710596616


En este apartado se ha aprendido a implementar los diferentes métodos explicados en clase para conjuntos de datos multietiquetas. Tanto de transformación como de adaptación. 